In [1]:
import sys
sys.path.append("./")
from my_class import *
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

Using TensorFlow backend.


In [2]:
train = pd.read_csv("./train.csv")
train = train.dropna(subset=["賃料"])
train = train.drop(20926)
temp = train["賃料"].values
threshold = 300000
is_rich_label = [0 for i in range(len(temp))]
for i in range(len(temp)):
    if temp[i] > threshold:
        is_rich_label[i] = 1
class_train = train.drop("賃料",axis = 1)
class_train = class_train.assign(target=is_rich_label)
class_train.head(1)
ct_x = class_train.drop("target",axis = 1)
ct_y = class_train["target"]

x_train,x_valid,y_train,y_valid = train_test_split(ct_x,ct_y,random_state=7778)

In [3]:
pre_step = [
    ("drop_id",drop_id()),
    ("parse_area",parse_area_size()),
    ("parse_room",parse_rooms()),
    ("parse_old",parse_how_old()),
    ("str_label",structure_label_encoder()),
    ("height_enc",height_encoder()),
    ("dir_enc",direction_encoder()),
    ("ex_dist",extract_district()),
    ("label_dist",district_encoder()),
    ("acc_ext",access_extractor()),
    ("train_enc",train_encoder()),
    ("bath_encoder",bath_encoder()),
    ("kit_encoder",kitchin_encoder()),
    ("fac_encoder",fac_encoder()),
    ("info_encoder",info_encoder()),
    ("parking_encoder",parking_encoder()),
    ("env_encoder",env_encoder()),
    ("drop_unnecessary",drop_unnecessary()),
]


m1 = [
    ("pre", Pipeline(steps = pre_step)),
    ("xgb",xgb.XGBRegressor(max_depth=8,min_child_weight=0))
]
m2 = [
    ("pre", Pipeline(steps = pre_step)),
    ("rfr",RandomForestRegressor())
]
m3 = [
    ("pre", Pipeline(steps = pre_step)),
    ("lgi",LogisticRegression())
]

boost = Pipeline(steps=m1)
forest = Pipeline(steps = m2)
logistic = Pipeline(steps = m3)

In [4]:
def score(model,x_valid,y_valid):
    temp = model.predict(x_valid)
    print(mean_absolute_error(temp,y_valid))

In [5]:
boost.fit(x_train,y_train)
score(boost,x_valid,y_valid)

/Users/kakeru/PycharmProjects/main_env/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:20:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.011252230911098523


In [6]:
forest.fit(x_train,y_train)
score(forest,x_valid,y_valid)

/Users/kakeru/PycharmProjects/main_env/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.012608032536858159


In [7]:
logistic.fit(x_train,y_train)
score(logistic,x_valid,y_valid)

/Users/kakeru/PycharmProjects/main_env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.009659379766141332


In [8]:
def ensemble(models,x_valid,y_valid):
    p = []
    predict = [0 for i in range(len(x_valid))]
    for m in models:
        p.append(m.predict(x_valid))
    for i in range(len(x_valid)):
        temp = 0
        for j in range(len(models)):
            temp += p[j][i]/3
        if temp > 0.5:
            predict[i] = 1
        else:
            predict[i] = 0
    predict = np.array(predict,dtype=np.int64)
    print(accuracy_score(predict,y_valid))

In [9]:
ensemble([boost,forest,logistic],x_valid,y_valid)

0.9931367564819522


In [10]:
ensemble([boost,forest,logistic],x_valid,y_valid)

0.9931367564819522


In [115]:
ensemble([forest],x_valid,y_valid)

0.9142094560244026


In [116]:
ensemble([logistic],x_valid,y_valid)

0.9142094560244026


In [117]:
ensemble([boost],x_valid,y_valid)

0.9142094560244026
